<a href="https://colab.research.google.com/github/jyun0618/TAVE_MLab/blob/main/Chapter06/jiyun_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## #1
> 백만 개의 샘플을 가진 훈련 세트에서 (규제 없이) 훈련시킨 결정 트리의 깊이는 대략 얼마일까요?


In [24]:
import math
math.log2(1000000)
# 약 20개

19.931568569324174

## #2
> 한 노드의 지니 불순도가 보통 그 부모 노드보다 낮을까요, 아니면 높을까요? 일반적으로 낮거나 높을까요, 아니면 항상 낮거나 높을까요?

한 노드의 지니 불순도는 일반적으로 부모의 불순도보다 낮다.
자식의 지니불순도의 가중치 합이 최소화하는 방향으로 각 노드를 분할하는 CART 훈련 알고리즘의 비용 함수 때문

ABAAA -> AB / AAA 이 경우 부모의 지니 불순도보다 커짐

## #3
> 결정 트리가 훈련 세트에 과대적합되었다면 max_depth를 줄이는 것이 좋을까요?

모델에 제약을 가해 규제해야 하므로 max_depth를 낮추는 것이 좋음

## #4
> 결정 트리가 훈련 세트에 과소적합되었다면 입력 특성의 스케일을 조정하는 것이 좋을까요?

결정 트리는 훈련데이터의 스케일에 상관하지 않음

## #5
> 백만 개의 샘플을 가진 훈련 세트에 결정 트리를 훈련시키는 데 한 시간이 걸렸다면 천만 개의 샘플을 가진 훈련 세트에 결정 트리를 훈련시키는 데는 대략 얼마나 걸릴까요?

결정 트리 훈련의 계산 복잡도: n × m log(m)

10log(10^7)/log(10^6) 약 11.7시간

## #6
> 어떤 훈련 세트에서 결정 트리를 훈련하는 데 1시간이 걸리는 경우, 특성의 개수를 두 배로 늘리면 대략 어느 정도의 시간이 걸릴까요?

특성 개수가 두 배로 늘어나면 훈련 시간도 약 두 배로 늘어남

## #7
> 다음 단계를 따라 moons 데이터셋에 결정 트리를 훈련시키고 세밀하게 튜닝해보세요.

a. make_moons(n_samples=1000, noise=0.4)를 사용해 데이터셋을 생성합니다.

In [10]:
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)

(array([[ 0.9402914 ,  0.12230559],
        [ 0.12454026, -0.42477546],
        [ 0.26198823,  0.50841438],
        ...,
        [-0.24177973,  0.20957199],
        [ 0.90679645,  0.54958215],
        [ 2.08837082, -0.05050728]]),
 array([1, 0, 0, ..., 1, 0, 1]))

b. 이를 train_test_split()을 사용해 훈련 세트와 테스트 세트로 나눕니다

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

array([0, 0, 1, ..., 1, 1, 0])

c. DecisionTreeClassifier의 최적의 매개변수를 찾기 위해 교차 검증과 함께 그리드 탐색을 수행합니다(GridSearchCV를 사용하면 됩니다).

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, cv=3, n_jobs=-1, verbose=1)

grid_search_cv.fit(X_train, y_train) # 찾은 매개변수를 사용해 전체 훈련 세트에 대해 모델 훈련

Fitting 3 folds for each of 294 candidates, totalling 882 fits


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16, 17, 18, 19, 20, 21,
                                            22, 23, 24, 25, 26, 27, 28, 29, 30,
                                            31, ...],
                         'min_samples_split': [2, 3, 4]},
             verbose=1)

In [5]:
grid_search_cv.best_estimator_

DecisionTreeClassifier(max_leaf_nodes=17, random_state=42)

d. 찾은 매개변수를 사용해 전체 훈련 세트에 대해 모델을 훈련시키고 테스트 세트에서 성능을 측정합니다. 대략 85~87%의 정확도가 나올 것입니다.

In [6]:
from sklearn.metrics import accuracy_score

y_pred = grid_search_cv.predict(X_test)
accuracy_score(y_test, y_pred)

0.8695

## #8.
> 다음 단계를 따라 랜덤 포레스트를 만들어보세요.

a. 이전 연습문제에 이어서, 훈련 세트의 서브셋을 1,000개 생성합니다. 각각은 무작위로 선택된 100개의 샘플을 담고 있습니다. 힌트: 사이킷런의 ShuffleSplit을 사용할 수 있습니다.

In [15]:
from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100

mini_sets = []

rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42) # mini-training 세트의 크기를 n_instances로 고정하고 나머지는 버리는 역할
for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

array([[ 1.729106  , -0.30293153],
       [ 0.82285486, -0.24639912],
       [ 0.6172433 ,  0.341603  ],
       [ 2.21942815, -0.52213898],
       [ 1.89842913, -0.45812873],
       [-0.91384812,  0.82723748],
       [ 0.12249021,  0.2451142 ],
       [ 0.52257396, -0.44650746],
       [-0.30268174,  1.2973055 ],
       [ 0.51989521,  0.26533397],
       [-0.71216241,  1.1853187 ],
       [ 2.40209241,  0.63410738],
       [ 0.75343015,  1.15911574],
       [ 1.0387203 , -0.62341768],
       [ 2.60009684,  0.32119377],
       [ 0.42509888,  0.13307243],
       [ 0.54796773,  0.95766374],
       [-0.79793684,  0.41911282],
       [ 1.56062298, -0.35677096],
       [ 1.25188949, -0.10284486],
       [ 1.89965546,  0.10692667],
       [ 0.600473  , -0.2956652 ],
       [-0.78906314,  0.62816483],
       [-0.26112405,  1.08783354],
       [ 1.10829435, -0.87313297],
       [ 0.74895939,  0.24851141],
       [ 0.2190207 , -0.01812386],
       [-0.43235994,  0.92969625],
       [-0.17739574,


b. 앞에서 찾은 최적의 매개변수를 사용해 각 서브셋에 결정 트리를 훈련시킵니다. 테스트 세트로 이 1,000개의 결정 트리를 평가합니다. 더 작은 데이터셋에서 훈련되었기 때문에 이 결정 트리는 앞서 만든 결정 트리보다 성능이 떨어져 약 80%의 정확도를 냅니다.

In [9]:
import numpy as np
from sklearn.base import clone

forest = [clone(grid_search_cv.best_estimator_) for _ in range(n_trees)] # n_trees 개수만큼 best_estimator 복제하여 리스트로 저장

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train) # forest에 저장한 best_estimator로 각각 훈련

    y_pred = tree.predict(X_test) # test셋 예측
    accuracy_scores.append(accuracy_score(y_test, y_pred))

np.mean(accuracy_scores) # 각각 샘플로 예측한 정확도 평균

np.float64(0.805471)

c. 이제 마술을 부릴 차례입니다. 각 테스트 세트 샘플에 대해 1,000개의 결정 트리 예측을 만들고 다수로 나온 예측만 취합니다(사이파이의 mode() 함수를 사용할 수 있습니다). 그러면 테스트 세트에 대한 _다수결 예측_이 만들어집니다.

In [17]:
Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forest): # Y_pred[i]는 i번째 트리의 X_test에 대한 예측값을 담고 있음
    Y_pred[tree_index] = tree.predict(X_test)

2000

In [19]:
from scipy.stats import mode
# mode() 함수는 axis=0을 기준으로 각 열(즉, 각 테스트 샘플의 예측값들)에서 가장 많이 등장한 값을 찾음
y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

# y_pred_majority_votes: 최빈값(가장 많이 나온 예측값, 최종 예측값)
# n_votes: 해당 최빈값이 등장한 횟수(가중치 없이 단순 빈도)

array([1, 1, 0, ..., 0, 0, 0])

d. 테스트 세트에서 이 예측을 평가합니다. 앞서 만든 모델보다 조금 높은(약 0.5~1.5% 정도) 정확도를 얻게 될 것입니다. 축하합니다. 랜덤 포레스트 분류기를 훈련시켰습니다!

In [20]:
accuracy_score(y_test, y_pred_majority_votes)

0.872